In [30]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk
n = lk('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/nfet_03v3.mat')
p = lk('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/pfet_03v3.mat')
GBW = 2e6
CL = 20e-12
Av_target = 10000  # 80dB
Vcm = 1.65
Rout_spec = 500
VDD = 3.3
VSS = 0
gm_id0 = 12  # NMOS bias
gm_id1 = 12  # NMOS input
gm_id2 = 12  # PMOS mirror
l0 = 0.5
l1 = 1.0  # longer for higher gain
l2 = 2.0  # longer for higher gain
l3 = 0.5
l4 = 0.5
k = 2.2  # multiplier for p2 = k * GBW
Cc_factor = 0.22
Cc = Cc_factor * CL
gm1 = 2 * np.pi * GBW * Cc
Id1 = gm1 / gm_id1
Id_tail = 2 * Id1
jd1 = n.lookup('ID_W', GM_ID=gm_id1, L=l1)
w1 = Id1 / jd1
jd2 = p.lookup('ID_W', GM_ID=gm_id2, L=l2)
w2 = Id1 / jd2
jd0 = n.lookup('ID_W', GM_ID=gm_id0, L=l0)
w0b = Id_tail / jd0
w0a = Id1 / jd0
gm3_target = 2 * np.pi * k * GBW * CL
gm_id3 = 15.0  # start high for high Rout
gm_id3_min = 10.0
step = 0.1
Rout = 1e12  # init high
found = False
while gm_id3 >= gm_id3_min:
    gm3 = gm3_target
    Id3 = gm3 / gm_id3
    jd3 = p.lookup('ID_W', GM_ID=gm_id3, L=l3)
    w3 = Id3 / jd3
    jd4 = n.lookup('ID_W', GM_ID=gm_id0, L=l4)
    w4 = Id3 / jd4
    gds_w3 = p.lookup('GDS_W', GM_ID=gm_id3, L=l3)
    gds3 = w3 * gds_w3
    gds_w4 = n.lookup('GDS_W', GM_ID=gm_id0, L=l4)
    gds4 = w4 * gds_w4
    g_o2 = gds3 + gds4
    Rout = 1 / g_o2 if g_o2 > 0 else 1e12
    if Rout <= Rout_spec + 1:  # tol
        found = True
        break
    gm_id3 -= step
if not found:
    print("Could not reach the specified Rout. Consider shorter L3/L4 or lower gm_id3_min.")
gds_w1 = n.lookup('GDS_W', GM_ID=gm_id1, L=l1)
gds1 = w1 * gds_w1
gds_w2 = p.lookup('GDS_W', GM_ID=gm_id2, L=l2)
gds2 = w2 * gds_w2
Av1 = gm1 / (gds1 + gds2)
gm3 = gm_id3 * Id3
Av2 = gm3 / g_o2
Av = Av1 * Av2
p2 = gm3 / (2 * np.pi * CL)
z = gm3 / (2 * np.pi * Cc)
cgg2 = w2 * p.lookup('CGG_W', GM_ID=gm_id2, L=l2)
cdd2 = w2 * p.lookup('CDD_W', GM_ID=gm_id2, L=l2)
cdd1 = w1 * n.lookup('CDD_W', GM_ID=gm_id1, L=l1)
gm2 = gm_id2 * Id1
fp_mirror = gm2 / (2 * cgg2 + cdd1 + cdd2) / (2 * np.pi)
cgd1 = w1 * n.lookup('CGD_W', GM_ID=gm_id1, L=l1)
fz_cgd = gm1 / cgd1 / (2 * np.pi) if cgd1 > 0 else 1e12
phip2 = -np.arctan(GBW / p2) * 180 / np.pi
phiz = -np.arctan(GBW / z) * 180 / np.pi
phip_mirror = -np.arctan(GBW / fp_mirror) * 180 / np.pi
phiz_cgd = -np.arctan(GBW / fz_cgd) * 180 / np.pi
PM = 90 + phip2 + phiz + phip_mirror + phiz_cgd
df = pd.DataFrame([Id1*1e6, gm1*1e3, Id3*1e6, gm3*1e3, Rout, Av, PM],
                  index=['Id1 (uA)', 'gm1 (mS)', 'Id3 (uA)', 'gm3 (mS)', 'Rout (ohm)', 'Av', 'PM (deg)'],
                  columns=['Value'])
print(df.round(2))
wfing = 5.0
nf0a = np.ceil(w0a / wfing)
nf0b = np.ceil(w0b / wfing)
nf1 = np.ceil(w1 / wfing)
nf2 = np.ceil(w2 / wfing)
nf3 = np.ceil(w3 / wfing)
nf4 = np.ceil(w4 / wfing)
df_sizes = pd.DataFrame([(w0a, w0b, w1, w2, w3, w4), (l0, l0, l1, l2, l3, l4), (nf0a, nf0b, nf1, nf2, nf3, nf4)],
                        index=['w (um)', 'l (um)', 'nf'], columns=['M0a', 'M0b', 'M1', 'M2', 'M3', 'M4'])
print(df_sizes.round(2))
Id0a = Id1
Id0b = Id_tail
Id2 = Id1
Id4 = Id3
df_id = pd.DataFrame([(Id0a*1e6, Id0b*1e6, Id1*1e6, Id2*1e6, Id3*1e6, Id4*1e6)],
                     index=['ID (uA)'], columns=['M0a', 'M0b', 'M1', 'M2', 'M3', 'M4'])
print(df_id.round(2))
print(Cc)

Could not reach the specified Rout. Consider shorter L3/L4 or lower gm_id3_min.
                Value
Id1 (uA)         4.61
gm1 (mS)         0.06
Id3 (uA)        55.29
gm3 (mS)         0.55
Rout (ohm)  169614.40
Av           28142.72
PM (deg)        54.05
         M0a   M0b    M1     M2    M3     M4
w (um)  1.69  3.39  3.34  31.45  44.2  20.33
l (um)  0.50  0.50  1.00   2.00   0.5   0.50
nf      1.00  1.00  1.00   7.00   9.0   5.00
          M0a   M0b    M1    M2     M3     M4
ID (uA)  4.61  9.22  4.61  4.61  55.29  55.29
4.4e-12
